## 챗봇 예제(역할 부여)

In [ ]:
# 사전 설치 : pip install ollama
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyMuPDFLoader
from langchain_community.embeddings import OllamaEmbeddings
import ollama
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.retrievers import WikipediaRetriever
import warnings

In [ ]:
# 챗봇의 기본적 질문, 답변 역할
def ask_gemma(question):
    # ollama를 사용하여 모델로부터 응답 생성
    chatbot_role = "You are a helpful assistant."
    response = ollama.chat(model='gemma2', messages=[
        {"role": "system", "content": chatbot_role},  # 챗봇의 기본 역할 부여
        {"role": "user", "content": question}, # 질문
    ])

    return response['message']['content']

In [ ]:
question = "세계에서 가장 높은 산을 알려줘."
response = ask_gemma(question)
print(response)

## 챗봇 예제(Gradio 사용)

In [ ]:
# 사전 설치 : pip install gradio
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage
import gradio as gr

In [ ]:
# ChatOllama 모델 초기화
model = ChatOllama(model="gemma2", temperature=0.7, verbose=False)

In [ ]:
# 채팅 기록을 포함하여 응답을 생성하는 함수
def chat(message, history):
    # 이전 대화 기록을 ChatOllama 형식으로 변환
    chat_history = []
    for human, ai in history:
        chat_history.append(HumanMessage(content=human))
        chat_history.append(AIMessage(content=ai))

    # 현재 메시지 추가
    chat_history.append(HumanMessage(content=message))

    # 모델을 사용하여 응답 생성
    response = model.invoke(chat_history)

    return response.content

In [ ]:
# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=chat,
    examples=[
        "안녕하세요!",
        "인공지능에 대해 설명해주세요.",
        "파이썬의 장점은 무엇인가요?"
    ],
    title="AI 챗봇",
    description="질문을 입력하면 AI가 답변합니다."
)

In [ ]:
# 서버 실행
demo.launch(server_port=7861, server_name="0.0.0.0")

In [ ]:
demo.close()

## 챗봇 예제(Gradio + csv 사용)

In [ ]:
import pandas as pd
from langchain_community.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
import gradio as gr

In [ ]:
# CSV 파일 로드
df = pd.read_csv("./dataset/indata_kor.csv", encoding='CP949')

In [ ]:
# 텍스트 분할
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text("\n".join(df.to_string()))

In [ ]:
# 임베딩 모델 초기화
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# 벡터 데이터베이스 생성
vectorstore = FAISS.from_texts(texts, embeddings)

In [ ]:
# ChatOllama 모델 초기화
llm = ChatOllama(model="gemma2", tempeature=0.1)

In [ ]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(search_kwargs={"k":1}),
    return_source_documents=True,
    verbose=False
)

In [ ]:
# 채팅 함수 정의
def chat(message, history):
    # 이전 대화 기록을 ConversationalRetrievalChain 형식으로 변환
    chat_history = [(human, ai) for human, ai in history]

    # 모델을 사용하여 응답 생성
    response = qa_chain({"question": message, "chat_history": chat_history})

    # 소스 문서 정보 추출
    sources = set([doc.metadata.get('source', 'Unknown') for doc in response['source_documents']])
    source_info = f"\n\n참고 출처: {', '.join(sources)}" if sources else ""

    return response['answer'] + source_info

In [ ]:
# Gradio 인터페이스 설정
demo = gr.ChatInterface(
    fn=chat,
    examples=[
        "한국폴리텍대학 스마트금융과 면접시에는 어떤걸 준비하고 가면 될까요?",
        "스마트금융과에 대해 설명해주세요",
        "한국폴리텍대한 추천할만한 학과 하나를 소개해주세요."
    ],
    title="대학 정보 AI 챗봇",
    description="스마트금융과에 대한 질문을 입력하면 AI가 CSV데이터를 참고하여 한글로 답변합니다."
)

In [ ]:
# 서버 실행
demo.launch(server_port=7861, server_name="0.0.0.0")

In [ ]:
demo.close()